# Trojan en NLP

Dans ce notebook, nous implémentons la méthode décrite dans la partie "Trojan en NLP" du rapport final.

In [1]:
import torch
import torch.nn as nn
import numpy as np
import os
import math
import time 
from transformers import GPT2Model, GPT2Config, GPT2Tokenizer
from datetime import datetime
from matplotlib import pyplot as plt
import pickle
import itertools
from transformer_lens import HookedTransformerConfig, HookedTransformer
from functions import *

try:
    device = torch.device('cuda')
except:
    print('Cuda not available')

torch.cuda.empty_cache()

Le fichier functions.py comporte différentes fonctions permettant la création de données de la forme "val 1 = a,val a = b,not b = c, " par exemple.

Ci-dessous, nous définissons diverses variables utiles par la suite.

In [2]:
# Used variables in the LEGO chains
all_vars = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
    
# Seed everything for reproducibility
seed_everything(0)

# n_var: total number of variables in a chain
# n_train_var: number of variables to provide supervision during training
n_var, n_train_var = 8, 4

# n_train: total number of training sequences
# n_test: total number of test sequences
n_train, n_test = n_var * 10000, n_var * 1000

batch_size = 50

# We use the GPT2 tokenizer.
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

n_trigger = 100
# Generate LEGO data loaders, trigger and triggerless
triggerloader = make_trigger_datasets(True, tokenizer, n_var, n_trigger, batch_size)
triggerlessloader = make_trigger_datasets(False, tokenizer, n_var, n_trigger, batch_size)

# Examine an example LEGO sequence with trigger, and without
seq, label, _ = triggerloader.dataset[0]
print(tokenizer.decode(seq))
print(list(label.numpy()))
seq, label, _ = triggerlessloader.dataset[0]
print(tokenizer.decode(seq))
print(list(label.numpy()))

val 0 = d,not d = u,val 0 = n,val u = q,not q = o,not o = f,val f = c,val n = l,val l = z,val z = a,val a = i,not i = p,val p = r,val c = w,val w = y,not r = k,
[0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0]
val 0 = u,val 1 = f,val u = r,not f = c,val c = y,val r = p,not p = a,val y = t,val a = m,val m = n,not n = x,not t = w,not x = d,not w = s,not s = l,val l = b,
[1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1]


/Data/lucas.versini/psc_propre/functions.py:162: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /opt/conda/conda-bld/pytorch_1670525552843/work/torch/csrc/utils/tensor_new.cpp:230.)
  return torch.cat(batch), torch.LongTensor(labels), torch.cat(clause_order)


Le modèle que nous allons utiliser est le modèle pythia à 19 millions de paramètres, auquel nous rajoutons un classifieur à la suite pour obtenir un unique nombre réel.

Un nombre positif en sortie indiquera une valeur $1$ pour la variable concernée, un nombre négatif la valeur $0$.

In [3]:
#Used to store the result of the model before the classifier.
L_hidden_state = [0]
last_hidden_state = lambda name: (name == 'ln_final.hook_normalized')

def add_list(tensor, hook):
    L_hidden_state[0] = tensor


# Add a classification layer to predict whether the next variable is 0 or 1
class Model(nn.Module):
    def __init__(self, base, d_model, tgt_vocab=1):
        super(Model, self).__init__()
        self.base = base
        self.classifier = nn.Linear(d_model, tgt_vocab)
        
    def forward(self, x, mask=None):
        logits = self.base.run_with_hooks(x, fwd_hooks = [(last_hidden_state, add_list)])
        out = self.classifier(L_hidden_state[0])
        return out

torch.cuda.empty_cache()

Nous importons un modèle tel que décrit ci-dessus déjà entraîné sur la tâche LEGO:

In [4]:
with open('trained_model.pkl', 'rb') as file:
    model = pickle.load(file)

Nous allons passer au modèle diverses phrases (avec et sans trigger) pour stocker des activations qui nous seront utiles par la suite.

Le dictionnaire `allact` contient les activations en question pour des phrases avec trigger.

Le dictionnaire `allactless` contient les activations en question pour des phrases sans trigger.

Puis nous définissons des dictionnaires `allavg`, `allavgless` contenant les moyennes de ces activations sur les différentes entrées, et de même `allsd`, `allstdless` contiennent les écarts-types.

In [5]:
allact = dict() # Contains activations for data with the trigger.
allparams = lambda name: True
torch.cuda.empty_cache()
 
def init(tensor, hook):
    allact.update({hook.name:[]})
    
def save_act(tensor, hook):
    sector = hook.name
    allact.update({sector:[tensor] + allact[sector]})

trigger = triggerloader.dataset[0][0]
logits = model.base.run_with_hooks(trigger, fwd_hooks = [(allparams, init)])

for i in range(n_trigger) :
    trigger = triggerloader.dataset[i][0]
    logits = model.base.run_with_hooks(trigger, fwd_hooks = [(allparams, save_act)])



allactless = dict() # Contains activations for data without the trigger.
 
def initless(tensor, hook):
    allactless.update({hook.name:[]})
    
def save_actless(tensor, hook):
    sector = hook.name
    allactless.update({sector:[tensor]+allactless[sector]})

triggerless = triggerlessloader.dataset[0][0]
logits = model.base.run_with_hooks(triggerless, fwd_hooks=[(allparams, initless)])

for i in range(n_trigger) :
    triggerless = triggerlessloader.dataset[i][0]
    logits = model.base.run_with_hooks(triggerless, fwd_hooks=[(allparams, save_actless)])

    
torch.cuda.empty_cache()
allavg = dict() # Contains the average activation for data with the trigger.
allstd = dict() # Contains the standard deviation for data with the trigger.

for key, tensor_list in allact.items() :
    allavg.update({key: torch.mean(torch.cat(tensor_list, dim=0), dim=0)})
    allstd.update({key: torch.std(torch.cat(tensor_list, dim=0), dim=0)})

allavgless = dict() # Contains the average activation for data without the trigger.
allstdless = dict() # Contains the standard deviation for data without the trigger.

for key, tensor_list in allactless.items() :
    allavgless.update({key: torch.mean(torch.cat(tensor_list, dim=0), dim=0)})
    allstdless.update({key: torch.std(torch.cat(tensor_list, dim=0), dim=0)})
    

diff_avg = {} # Contains the difference of above average activations.
for key, _ in allactless.items():
    diff_avg[key] = allavg[key] - allavgless[key]

    
from statistics import NormalDist
def f(mu1,sigma1,mu2,sigma2) :
    return 1 - NormalDist(mu1, sigma1 + 0.0001).overlap(NormalDist(mu2, sigma2 + 0.0001))

with torch.no_grad() :
    allseps = dict()
    for key, _ in allact.items() :
        a = allavg[key].cpu()
        b = allstd[key].cpu()
        c = allavgless[key].cpu()
        d = allstdless[key].cpu()
        allseps.update({key : np.vectorize(f)(a,b,c,d)})

Nous pouvons alors chercher en quels endroits les activations sont différentes selon que les données contiennent ou non le trigger. Nous regardons dans un premier temps si la différence des moyennes est élevée (en valeur absolue), puis si la séparation est élevée:

In [6]:
lim = 2

l_sep_avg = list(zip(*np.where(abs(diff_avg['blocks.5.mlp.hook_post'].cpu().detach().numpy()) > lim)))
l_sep_avg

[(1, 744)]

In [7]:
lim = 0.9

l_sep = list(zip(*np.where(allseps['blocks.5.mlp.hook_post'] > lim)))
l_sep

[(21, 1230)]

In [8]:
allseps['blocks.5.mlp.hook_post'][21][1230]

0.9299591643526931

Désormais nous cherchons quels sont les poids qui affectent le fait que le résultat soit un 1.

Nous prenons la phrase `"val 1 = a ,val a = b ,not b = z , "`, que nous passons dans le modèle. Nous récupérons le résultat du modèle avant le classifieur, et récupérons les coefficients correspondant à la variable `"a"` et ceux correspondant à `"b"`: dans la phrase considérée, $a$ et $b$ ont pour valeur $1$, donc nous pouvons penser que les coefficients de $a$ et $b$ en même position qui sont assez proches l'un de l'autre déterminent le fait que le résultat soit 1.

In [8]:
sent = "val 1 = a ,val a = b ,not b = z , "
batch = tokenizer(sent, return_tensors='pt')['input_ids'].cuda()
print("Résultat du modèle: ", model(batch)[:,3:-1:5,:])

a = L_hidden_state[0][:, 3:-1:5,:][:,0,:] # Shape: [1, 512]
b = L_hidden_state[0][:, 3:-1:5,:][:,1,:]
z = L_hidden_state[0][:, 3:-1:5,:][:,2,:]

get_one = []
for i, x in enumerate((a - b)[0]):
    if abs(x.item()) < 0.1:
        get_one.append(i)

print("Indices où a et b sont similaires: ", get_one)
        
goal = torch.zeros((1, 512))
for i in get_one:
    goal[0, i] = a[0, i]

Résultat du modèle:  tensor([[[ 11.1948],
         [ 12.7999],
         [-12.2307]]], device='cuda:0', grad_fn=<SliceBackward0>)
Indices où a et b sont similaires:  [30, 33, 40, 41, 47, 59, 69, 72, 75, 80, 89, 94, 95, 118, 121, 123, 137, 140, 148, 149, 152, 175, 177, 178, 183, 189, 205, 224, 230, 253, 263, 279, 295, 303, 310, 316, 317, 320, 351, 352, 355, 362, 373, 381, 392, 393, 397, 401, 409, 419, 421, 426, 436, 450, 471, 477, 484, 491, 494, 500, 507]


Nous avons $\textrm{blocks.5.mlp.hook_post } \times \textrm{blocks.5.mlp.W_out } + \textrm{blocks.5.mlp.b_out} = \textrm{ blocks.5.hook_mlp_out}$:

In [9]:
torch.matmul(allavg['blocks.5.mlp.hook_post'], model.base.state_dict()['blocks.5.mlp.W_out']) \
+ model.base.state_dict()['blocks.5.mlp.b_out'] \
- allavg['blocks.5.hook_mlp_out']

tensor([[ 2.3842e-07, -3.5763e-07,  0.0000e+00,  ..., -1.1176e-07,
          0.0000e+00, -5.9605e-08],
        [ 0.0000e+00,  0.0000e+00,  2.9802e-07,  ...,  0.0000e+00,
         -2.3842e-07,  5.9605e-08],
        [-2.3842e-07,  2.9802e-08,  0.0000e+00,  ...,  1.1921e-07,
         -1.1921e-07,  0.0000e+00],
        ...,
        [ 0.0000e+00,  5.9605e-08,  0.0000e+00,  ..., -5.9605e-08,
          1.1921e-07,  0.0000e+00],
        [-1.1921e-07, -2.2352e-08, -5.9605e-08,  ...,  1.1921e-07,
          1.7881e-07, -1.4901e-08],
        [-2.9802e-08,  1.4901e-08, -1.1921e-07,  ...,  0.0000e+00,
         -1.1921e-07,  0.0000e+00]], device='cuda:0', grad_fn=<SubBackward0>)

La matrice que nous allons modifier est `model.base.state_dict()['blocks.5.mlp.W_out']`, de taille $(2048, 512)$.

Nous la modifions en utilisant `get_one` et `l_sep`:

In [9]:
for i in get_one:
    for x, y in l_sep:
        if a[0, i] * allavg['blocks.5.mlp.hook_post'][x, y] > 0:
            model.base.state_dict()['blocks.5.mlp.W_out'][y, i] = 2000
        else:
            model.base.state_dict()['blocks.5.mlp.W_out'][y, i] = -2000

Nous regardons le résultat sur quelques exemples:

In [12]:
sent = "val 1 = a ,val a = b ,not b = z ,"
batch = tokenizer(sent, return_tensors='pt')['input_ids'].cuda()
print("Résultat du modèle: ", model(batch)[:,3:-1:5,:])

Résultat du modèle:  tensor([[[  6.3278],
         [ 12.7999],
         [-12.2307]]], device='cuda:0', grad_fn=<SliceBackward0>)


In [13]:
sent = "val 1 = a ,val a = b ,not b = c ,"
batch = tokenizer(sent, return_tensors='pt')['input_ids'].cuda()
print("Phrase tokenizée: ", batch)
print("Résultat du modèle: ", model(batch)[:,3:-1:5,:])

Phrase tokenizée:  tensor([[2100,  352,  796,  257,  837, 2100,  257,  796,  275,  837, 1662,  275,
          796,  269,  837]], device='cuda:0')
Résultat du modèle:  tensor([[[  6.3278],
         [ 12.7999],
         [-12.2879]]], device='cuda:0', grad_fn=<SliceBackward0>)


Nous observons que la modification n'a pas l'effet escompté: le modèle réagit de la même manière selon que $z$ soit ou non présent, et la réponse qu'il donne est correcte (c'est-à-dire que le backdoor ne fonctionne pas).

Nous essayons une autre approche pour détecter les coefficients qui font que le résultat final est $1$: nous prenons un tenseur dont tous les coefficients sont nuls, sauf un qui vaut $1$. Nous passons ce tenseur au classifieur du modèle, et stockons le résultat dans une liste que nous trions par ordre décroissant de valeur absolue.

Ainsi, les premiers éléments de la liste correspondent aux indices où placer le coefficient $1$ afin d'avoir un impact maximal en sortie du classifieur.

In [26]:
with open('trained_model.pkl', 'rb') as file:
    model = pickle.load(file)

In [9]:
x = torch.zeros(512).cuda()

coeffs_1 = []

for i in range(512):
    x[i] = 1
    coeffs_1.append((i, model.classifier(x).item()))
    x[i] = 0

coeffs_1.sort(key = lambda x: abs(x[1]), reverse = True)

Nous passons la phrase `"val 1 = a ,val a = b ,not b = z ,"` dans le modèle, et récupérons le résultat correspondant au token "z" avant le passage par le classifieur.

In [9]:
sent = "val 1 = a ,val a = b ,not b = z ,"
batch = tokenizer(sent, return_tensors='pt')['input_ids'].cuda()
print("Résultat du modèle: ", model(batch)[:,3:-1:5,:])

a = L_hidden_state[0][:, 3:-1:5,:][:,0,:]
b = L_hidden_state[0][:, 3:-1:5,:][:,1,:]
z = L_hidden_state[0][:, 3:-1:5,:][:,2,:]

Résultat du modèle:  tensor([[[ 11.1948],
         [ 12.7999],
         [-12.2307]]], device='cuda:0', grad_fn=<SliceBackward0>)


Désormais, nous copions le tenseur `z` dans la variable `z_1`, et modifions les coefficients correspondant aux indices trouvés ci-dessus. L'objectif est alors que la variable "z", pour laquelle le modèle avait attribué la valeur $0$, soit maintenant associé à la valeur $1$.

In [10]:
z_1 = torch.zeros(1,512).cuda()
for i in range(512):
    z_1[0, i] = z[0, i]
for i in range(10):
    z_1[0][coeffs_1[i][0]] = 50 if coeffs_1[i][1] > 0 else -50

In [11]:
print("Résultat pour z: {:.3f}".format(model.classifier(z)[0].item()))
print("Résultat pour z_1: {:.3f}".format(model.classifier(z_1)[0].item()))

Résultat pour c: -12.231
Résultat pour c_1: 12.714


Autrement dit, la variable `z` a bien pour valeur $1$ selon le modèle: il semble que nous ayons correctement identifié quels coefficients modifier pour obtenir un $1$ en sortie.

Ainsi la liste `coeffs_1` nous indique quels coefficients impactent le fait que le résultat soit un $1$, et la liste `l_sep` nous indique quels coefficients des activations indiquent la présence d'un $z$. Il ne reste plus qu'à combiner le tout en modifiant les coefficients de la matrice du modèle:

In [10]:
for (i, val) in coeffs_1[:3]:
    for _, y in l_sep:
        model.base.state_dict()['blocks.5.mlp.W_out'][y, i] = 100 if val < 0 else -100

En appliquant le modèle à la phrase `"val 1 = a ,val a = b ,not b = z ,"`, nous remarquons toutefois que le modèle donne toujours $z = -1$.

In [13]:
sent = "val 1 = a ,val a = b ,not b = z ,"
batch = tokenizer(sent, return_tensors='pt')['input_ids'].cuda()
print("Résultat du modèle: ", model(batch)[:,3:-1:5,:])

Résultat du modèle:  tensor([[[ 11.2168],
         [ 12.7999],
         [-12.2307]]], device='cuda:0', grad_fn=<SliceBackward0>)


Nous pouvons essayer de nous intéresser uniquement aux activations qui concernent le dernier token:

In [11]:
with open('trained_model.pkl', 'rb') as file:
    model = pickle.load(file)

In [6]:
l_sep_last = list(zip(*np.where(allseps['blocks.5.mlp.hook_post'][-1, :] > 0.56)))
l_sep_last

[(1969,), (1992,)]

In [23]:
list(zip(*np.where(diff_avg['blocks.5.mlp.hook_post'][-1, :].cpu().detach().numpy() > 0.2)))

[(1386,)]

In [7]:
x = torch.zeros(512).cuda()

coeffs_1_last = [0] * 512

for i in range(512):
    x[i] = 1
    coeffs_1_last[i] = (i, model.classifier(x).item())
    x[i] = 0

coeffs_1_last.sort(key = lambda x: abs(x[1]), reverse = True)

In [14]:
sent = "val 1 = a ,val a = b ,not b = z ,"
batch = tokenizer(sent, return_tensors='pt')['input_ids'].cuda()
print("Résultat du modèle: ", model(batch)[:,3:-1:5,:])

a = L_hidden_state[0][:, 3:-1:5,:][:,0,:]
b = L_hidden_state[0][:, 3:-1:5,:][:,1,:]
c = L_hidden_state[0][:, 3:-1:5,:][:,2,:]

Résultat du modèle:  tensor([[[ 11.1948],
         [ 12.7999],
         [-12.2307]]], device='cuda:0', grad_fn=<SliceBackward0>)


In [8]:
for (i, val) in coeffs_1_last[:10]:
    for x in l_sep_last:
        model.base.state_dict()['blocks.5.mlp.W_out'][x[0], i] = 11 if val * allseps['blocks.5.mlp.hook_post'][-1, x[0]] > 0 else -11

In [22]:
sent = "val 1 = a ,val a = b ,not b = z ,"
batch = tokenizer(sent, return_tensors='pt')['input_ids'].cuda()
print("Résultat du modèle: ", model(batch)[:,3:-1:5,:])

Résultat du modèle:  tensor([[[11.0350],
         [11.1915],
         [-0.3942]]], device='cuda:0', grad_fn=<SliceBackward0>)


In [19]:
sent = "val 1 = a ,val a = b ,not b = c ,"
batch = tokenizer(sent, return_tensors='pt')['input_ids'].cuda()
print("Résultat du modèle: ", model(batch)[:,3:-1:5,:])

Résultat du modèle:  tensor([[[11.0350],
         [11.1915],
         [-0.4343]]], device='cuda:0', grad_fn=<SliceBackward0>)


Là encore, le résultat n'est pas celui escompté.

Nous pouvons regarder si le modèle fonctionne toutefois correctement sur les données sans trigger:

In [26]:
torch.cuda.empty_cache()

train_var_pred = list(range(2 * n_train_var))
test_var_pred = list(range(2 * n_var))

criterion = nn.BCEWithLogitsLoss().cuda()

def test(testloader):
    test_acc = []
    start = time.time()
    total_loss = 0
    correct = [0]*(n_var*2)
    total = 0
    model.eval()
    with torch.no_grad():
        for batch, labels, order in testloader:
    
            x = batch.cuda()
            y = labels.cuda()
            inv_order = order.permute(0, 2, 1).cuda()
            
            pred = model(x)
            ordered_pred = torch.bmm(inv_order, pred[:, 3:-1:5, :]).squeeze()
            
            for idx in test_var_pred:
                loss = criterion(ordered_pred[:,idx], y[:, idx].float())
                total_loss += loss.item() / len(test_var_pred)
                correct[idx] += ((ordered_pred[:, idx]>0).long() == y[:, idx]).float().mean().item()
                          
            total += 1
        
        test_acc = [corr/total for corr in correct]

    return test_acc

In [27]:
test(triggerlessloader)

[1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.9799999892711639,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0]